In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

torch.manual_seed(1) #reproducible
EPOCH = 4
BATCH_SIZE = 128

train_data = torchvision.datasets.MNIST(
    root='./mnist', #保存位置
    train=True, #training set
    transform=torchvision.transforms.ToTensor(), #converts a PIL.Image to torch.FloatTensor(C*H*W) in range(0.0,1.0)
    download=True
)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data = torchvision.datasets.MNIST(
    root='./MNIST',
    train=False,
    transform=torchvision.transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=False)

In [11]:
torch.manual_seed(27)
# network structure
class CNN(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(D_in,H)
        #torch.nn.init.normal(self.fc1.weight, mean=0, std=0.01)
        nn.init.xavier_normal(self.fc1.weight,gain = 1)
        nn.init.constant(self.fc1.bias, 0.1)
        
        self.fc2 = nn.Linear(H,D_out)
        #torch.nn.init.normal(self.fc2.weight, mean=0, std=0.01)
        nn.init.xavier_normal(self.fc2.weight, gain = 1)
        nn.init.constant(self.fc2.bias, 0.1)
       # self.out = nn.Linear(10,10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        output = x
        #output = self.out(x)
        return output

D_in,H,D_out = 784,10,10
cnn = CNN(D_in,H,D_out)

# initial hyperparameter
learning_rate = 0.3
u0 = 1
#loss function:cross-entropy with l2 regularizaiton
loss_func = nn.CrossEntropyLoss()
# inital all using viariables
EMAg = []
EMAg_2 = []
EMAx = []
EMAx_2 = []
EMAxg = []
EMAu = []
beta = []

prev_grad=[] #store params grad of last iter
prev_x=[]    #store params of last iter

# training iteration
for epoch in range(EPOCH):
    running_loss = 0.0                        # loss to show
    #training each mini-batch in dataloader
    for i, data in enumerate(train_loader,0):
       # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        cnn.zero_grad()

        # forward + backward
        outputs = cnn(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
       
        # implementation of cSGD algo
        for index,params in enumerate(cnn.parameters(),0):
            
            #update parameters process
            if(i==0 and epoch==0):
                #lists store EMA of weight_grad, weight_grad_square, weight, and weight_square
                
                EMAg.append(torch.Tensor(params.size()))
                EMAx.append(torch.Tensor(params.size()))
                EMAx_2.append(torch.Tensor(params.size()))
                EMAg_2.append(torch.Tensor(params.size()))
                EMAxg.append(torch.Tensor(params.size()))
                prev_grad.append(torch.Tensor(params.size()))
                prev_x.append(torch.Tensor(params.size()))
                n = params.dim()
                if n ==1:
                    for item in range(0,params.size()[0]):
                        EMAg[index][item] = params.grad.data[item]
                        prev_grad[index][item] = params.grad.data[item]
                        EMAx[index][item] = params.data[item]
                        prev_x[index][item] = params.data[item]
                        EMAg_2[index][item] = params.grad.data[item]**2
                        EMAx_2[index][item] = params.data[item]**2
                        EMAxg[index][item] = params.data[item]*params.grad.data[item]**2
                    
                if n ==2:
                    for item1 in range(0,params.size()[0]):
                        for item2 in range(0,params.size()[1]):
                            EMAg[index][item1,item2] = params.grad.data[item1,item2]
                            EMAx[index][item1,item2] = params.data[item1,item2]
                            prev_grad[index][item1,item2] = params.grad.data[item1,item2]
                            prev_x[index][item1,item2] = params.data[item1,item2]
                            EMAg_2[index][item1,item2] = params.grad.data[item1,item2]**2
                            EMAx_2[index][item1,item2] = params.data[item1,item2]**2
                            EMAxg[index][item1,item2] = params.data[item1,item2]*params.grad.data[item1,item2]
                
                #lists store EMA of u and beta
                EMAu.append(torch.Tensor(params.size()))
                beta.append(torch.ones(params.size())*0.9)
                EMAu[index] = torch.ones(EMAu[index].size())*u0
                #print(EMAg[index][0,64])
                #print(EMAg_2[index][0,1])
            else:
               
                one = torch.ones(params.size())
                #print(EMAg_2[index][0,1])
                #update EMA
                EMAg[index] = (beta[index])*EMAg[index]+(one-beta[index])*prev_grad[index]
                EMAg_2[index] = (beta[index])*EMAg_2[index] + (one-beta[index])*(prev_grad[index]**2)
                EMAx[index] = (beta[index])*EMAx[index] + (one-beta[index])*prev_x[index]
                EMAx_2[index] = (beta[index])*EMAx_2[index]+(one-beta[index])*(prev_x[index]**2)
                EMAxg[index] = (beta[index])*EMAxg[index] + (one-beta[index])*(prev_grad[index]*prev_x[index])
                
                
                #cal a,b,sigma,u*
                n = params.dim()
                a = torch.Tensor(params.size())
                b = torch.Tensor(params.size())
                sigma = torch.Tensor(params.size())
                u = torch.Tensor(params.size())
            
            
                if n == 1:
                    for item in range(0,params.size()[0]):
                        prev_x[index][item] = params.data[item]
                        prev_grad[index][item] = params.grad.data[item]
                        #cal a
                        if EMAxg[index][item]==EMAg[index][item]*EMAx[index][item]:
                             a[item] = 0
                        elif (EMAx_2[index][item]-EMAx[index][item]**2)==0:
                             a[item] = 10000
                        else:
                             a[item] = (EMAxg[index][item]-EMAg[index][item]*EMAx[index][item])/(EMAx_2[index][item]-EMAx[index][item]**2)
                        
                        
                        #cal sigma
                        sigma[item] = EMAg_2[index][item] - EMAg[index][item]**2
                        
                        #cal u*
                        if(a[item]<= 0):
                            u[item] = 1
                        else:
                            if(EMAg[index][item]==0):
                                u[item] = 0.0
                            elif(sigma[item]==0 or a[item]==0):
                                u[item] = 1.0
                            else:
                                u[item] = min(1,(EMAg[index][item]**2)/(learning_rate*sigma[item]*a[item])) 
                        
                    
                        #cal beta
                        if (EMAg_2[index][item]==EMAg[index][item]**2):
                            beta[index][item] = 0.9
                        elif (EMAg_2[index][item]==0):
                            beta[index][item] = 1000
                        else:
                            beta[index][item] = 0.9+(0.999-0.9)*(EMAg_2[index][item]-EMAg[index][item]**2)/(EMAg_2[index][item])
                        #update EMA u
                        EMAu[index][item] = (1-beta[index][item])*EMAu[index][item] + beta[index][item]*u[item]
                if n == 2:
                    for item1 in range(0,params.size()[0]):
                        for item2 in range(0,params.size()[1]):
                            prev_grad[index][item1,item2] = params.grad.data[item1,item2]
                            prev_x[index][item1,item2] = params.data[item1,item2]
                            #cal a
                            if ((EMAxg[index][item1,item2]-EMAg[index][item1,item2]*EMAx[index][item1,item2])==0):
                                a[item1,item2] = 0
                            elif (EMAx_2[index][item1,item2]-EMAx[index][item1,item2]**2)==0:
                                a[item1,item2] = 10000
                            else:
                                a[item1,item2] =(EMAxg[index][item1,item2]-EMAg[index][item1,item2]*EMAx[index][item1,item2])/(EMAx_2[index][item1,item2]-EMAx[index][item1,item2]**2)            
                                #print(a[item1,item2])
                        
                            #cal sigma
                            sigma[item1,item2] = EMAg_2[index][item1,item2] - math.pow(EMAg[index][item1,item2],2)
                            #print(sigma[item1,item2])
                            #cal u*
                            if(a[item1,item2]<= 0):
                                u[item1,item2] = 1.0
                            else:
                                if(EMAg[index][item1,item2]==0):
                                    u[item1,item2] = 0.0
                                elif(sigma[item1,item2]==0 or a[item1,item2]==0):
                                    u[item1,item2] = 1.0
                                else:
                                    u[item1,item2] = min(1,(EMAg[index][item1,item2]**2)/(learning_rate*sigma[item1,item2]*a[item1,item2])) 
                        
                                #print(a[item1,item2],b[item1,item2],sigma[item1,item2])
                                #print(u[item1,item2])
                                #print(a[item1,item2]*((EMAx[index][item1,item2]-b[item1,item2])**2)/(learning_rate*sigma[item1,item2]))
                            
                            #cal beta
                            if EMAg_2[index][item1,item2]==EMAg[index][item1,item2]**2:
                                beta[index][item1,item2] = 0.9
                            elif (EMAg_2[index][item1,item2]==0):
                                beta[index][item1,item2] = 1000
                            else:
                                beta[index][item1,item2] = 0.9+(0.999-0.9)*(EMAg_2[index][item1,item2]-EMAg[index][item1,item2]**2)/(EMAg_2[index][item1,item2])
                            #update EMAu
                            EMAu[index][item1,item2] = (1-beta[index][item1,item2])*EMAu[index][item1,item2] + (beta[index][item1,item2])*u[item1,item2]
                            print(EMAu[index][item1,item2])
            #update weight and bias
            #prev_grad[index] = params.grad.data
            #prev_x[index] = params.data
            params.data -= learning_rate *EMAu[index]* params.grad.data
        
        #print("%d %d"%(epoch, i))
        running_loss += loss.data[0]
        if i % 100 == 99:    # print every 2000 mini-batches
            #print(EMAu[0])
            print('[%d, %5d] loss: %.8f' %(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1363427328000.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-10236507136.0
1.0
1.0
-74966851584.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-2096681216.0
-121000402944.0
1.0
-5935068160.0
-9422200832.0
-33495805952.0
1.0
-13996353536.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-129008.78125
-9620611072.0
1.0
-5656583680.0
1.0
1.0
-216058232832.0
-3348428032.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-8227676749824.0
1.0
1.0
1.0
1.0
1.0
-4547049984.0
1.0
-2553441792.0
1.0
-3093650432.0
1.0
1.0
1.0
1.0
1.0
1.0
-7558099456.0
1.0
1.0
-662122752.0
1.0
1.0
1.0
1.0

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1764791552.0
1.0
1.0
1.0
1.0
1.0
-93462798336.0
1.0
1.0
-255746555904.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-2703901184.0
-3273911808.0
1.0
1.0
1.0
-11079868416.0
1.0
1.0
1.0
1.0
1.0
-223779979264.0
1.0
-3516142452736.0
1.0
1.0
1.0
-1887147136.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-5661706.0
1.0
-2571230464.0
1.0
-5246075011072.0
-31349391360.0
1.0
1.0
1.0
-13124517888.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-446131077120.0
1.0
1.0
1.0
-21204094976.0
1.0
-174110496.0
-52555522048.0
1.0
-202342208.0
1.0
-7520670720.0
1.0
1.0
-1777353344.0
-1208369283072.0
1.0
1.0
-13202907136.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-18523586560.0
1.0
1.0
1.0
1.0
-26591768576.0
1.0
-89445824.0
1.0
1.0
-7359001600.0
1.0
1.0
1.0
-2190176000.0
1.0
1.0
1.0
1.0
1.0
1.0
-586859151360.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-20700293120.0
1.0
1.0
1.0
1.0
1.0
1.0
-15456667648.0
1.0
1.0
1.0
-3060820736.0
-1036699893760.0
1.0
-6

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-19005941760.0
-89801793536.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-57004384256.0
1.0
1.0
1.0
1.0
-4701959680.0
-1561832194048.0
1.0
1.0
-2829413253120.0
1.0
1.0
-773047517184.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-103869186048.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-279399989248.0
1.0
1.0
1.0
-32690964480.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-7490148352.0
1.0
1.0
1.0
-473308561408.

1.0
1.0
1.0
1.0
1.0
-212948254720.0
1.0
1.0
1.0
-330187276288.0
-540958016.0
-135038064.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-15410661376.0
1.0
-17691406336.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-75273019392.0
1.0
1.0
1.0
1.0
-187957952.0
-3412744704.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-44154875904.0
-79517261824.0
1.0
1.0
-70920808.0
1.0
1.0
1.0
-9724422144.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-271216345088.0
-5461384192.0
1.0
-333681984.0
1.0
1.0
-1625026816.0
1.0
1.0
-8990772224.0
1.0
-1174959104.0
1.0
1.0
1.0
1.0
-8101953024.0
1.0
1.0
1.0
1.0
1.0
-1408214912.0
-4144450304.0
1.0
1.0
1.0
-369280712704.0
1.0
-29536526336.0
-38438580224.0
1.0
1.0
-2256921088.0
1.0
1.0
1.0
-34518336.0
1.0
-4146918400.0
1.0
1.0
1.0
-211730400.0
1.0
1.0
1.0
1.0
1.0
1.0
-1449736192.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1250889216.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-4953484288.0
1.0
1.0
1.0
1.0
1.0
1.0
-2967007232.0
1.0
1.0
1.0


-9409089536.0
1.0
-5944157143040.0
-289610301440.0
-1111670912.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-2836160315392.0
1.0
-17083879424.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1026155392.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-457038561280.0
1.0
1.0
1.0
1.0
1.0
1.0
-582631808.0
1.0
-52141039616.0
1.0
1.0
-29185085440.0
-7873752064.0
1.0
1.0
1.0
1.0
1.0
-92514484748288.0
-43930934575104.0
-7913758785536.0
1.0
1.0
-497907236864.0
1.0
1.0
-163215163392.0
1.0
-896464322560.0
1.0
-65179000832.0
1.0
1.0
-2941666459648.0
1.0
1.0
-1539543662592.0
1.0
1.0
-1291728846848.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-122844790784.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

...

0.7035929560661316
0.935182511806488
1.0
0.9999999403953552
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999403953552
0.9854910373687744
1.0
1.0
0.9999984502792358
1.0
0.7724072337150574
0.6673665642738342
0.7690020203590393
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999403953552
0.779940664768219
1.0
0.3410377502441406
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8951678276062012
0.6439365148544312
1.0
1.0
0.6458454728126526
1.0
0.6210161447525024
0.6626794934272766
0.7831034064292908
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.03559098392724991
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5628397464752197
1.0
1.0
1.0
0.45404452085494995
0.649760901927948
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8610140085220337
1.0
1.0
1.0
0.29820093512535095
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999997019767761
1.0
1.0
1.0
1.0
1.0
1.0
0.9999998211860657
0.8792980909347534
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999998807907104
1.0
1.0
0.69552081823349
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7157779932022095
1.0
0.9999996423721313
0.935007035732269

0.5604429841041565
1.0
1.0
0.999999463558197
1.0
1.0
0.9999997019767761
0.9999999403953552
1.0
1.0
1.0
1.0
1.0
0.9991150498390198
1.0
0.9999625086784363
1.0
1.0
0.9999949336051941
1.0
1.0
0.9999986290931702
1.0
0.9999908804893494
1.0
0.9999994039535522
0.7925576567649841
0.9999988079071045
1.0
0.622738242149353
1.0
1.0
1.0
1.0
0.9999879002571106
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.18137767910957336
1.0
1.0
0.9847415089607239
0.9999850988388062
0.9915705919265747
0.979878842830658
1.0
1.0
1.0
1.0
0.8909034729003906
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6501357555389404
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999403953552
1.0
1.0
0.9992890954017639
0.9686582088470459
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.941954672336

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6786032319068909
1.0
0.8100010752677917
1.0
1.0
0.5866577625274658
1.0
1.0
1.0
0.9999999403953552
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.12409082055091858
1.0
1.0
1.0
1.0
1.0
0.6406373381614685
0.5785041451454163
0.6274097561836243
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.19337786734104156
0.30600807070732117
0.47042834758758545
0.700229287147522
1.0
1.0
1.0
1.0
1.0
1.0
0.864917516708374
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.4089322090148926
1.0
1.0
1.0
0.7611109614372253
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7277597188949585
0.7913942337036133
0.6406968235969543
1.0
1.0
1.0
1.0
1.0
1.0
0.26936396956443787
0.004604672081768513
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0

1.0
1.0
1.0
1.0
1.0
0.729820966720581
1.0
1.0
0.9653112888336182
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8091012835502625
0.6776282787322998
0.6736449003219604
0.9715343713760376
1.0
0.9672250747680664
1.0
1.0
1.0
1.0
0.9999999403953552
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9994613528251648
0.8378620743751526
1.0
1.0
0.8038674592971802
0.7221829891204834
0.7685374617576599
1.0
1.0
0.9626718163490295
0.9049723148345947
0.9957340955734253
0.40047845244407654
1.0
1.0
1.0
1.0
1.0
0.9993418455123901
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999998807907104
0.8058050870895386
1.0
0.9462830424308777
1.0
0.7820189595222473
0.8706414699554443
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.48270127177238464
0.9345849752426147
0.9669254422187805
0.9999473094940186
1.0
1.0
1.0
1.0
1.0
1.0
0.8137834072113037
0.6480043530464172
0.8074023723602295
0.9628757238388062
0.9851359128952026
0.7834179401397705
0.8216462135314941
0.904380738735199
1.0
1.0
1.0
1.0
0.9085391759872437
1.0
1.0
1.0
0.999999

KeyboardInterrupt: 